In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [ ]:
global list_of_dataframes 
list_of_dataframes = []


html_element = "table.outlinetable:nth-child(10)"
page = requests.get("https://cloford.com/resources/codes/index.htm")
soup = BeautifulSoup(page.text, 'html.parser')

tables = soup.find_all('table',class_='outlinetable')
print(f'There were {len(tables)} tables found:\n')

for each_table in tables:
    translate_table_to_dataframe(each_table)

names = ["Country Codes", "Additional Codes", "Additional FIPS 10-4 Codes"]
for one_dataframe, one_name in zip(list_of_dataframes, names):
    one_dataframe.name = one_name
    print(one_dataframe.name)

In [ ]:
def translate_table_to_dataframe(table):
    dataframe_friendly_array = []
    dataframe_ready_headers = []
    column_headers = table.find_all('th')
    for header in column_headers:
        dataframe_ready_headers.append(header.text)

    rows = table.find_all('tr')

    for row in rows:
        row_data = []
        for col in row.find_all('td'):
            try:
                row_data.append(col.text)
            except:
                continue
        dataframe_friendly_array.append(row_data)
            
    dataframe = pd.DataFrame(data = dataframe_friendly_array, columns = dataframe_ready_headers)
    list_of_dataframes.append(dataframe)

In [ ]:
df_list = [list_of_dataframes[i] for i in range(3)]

final = pd.concat(df_list).drop(index=0)

remove_list = ['-', '--', '\xa0', np.nan]

for column in final.columns:
    
    final.loc[final[column].isin(remove_list), column] = 'No Record'

final.to_csv('GeoInformation_Country.csv', index = False)

final